from bs4 import BeautifulSoup
import requests
import pymongo
import pandas as pd

In [55]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo
import pandas as pd
import time
import os
from splinter import Browser

In [56]:
#path to chromedriver
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [57]:
#Nasa Mars News
url = "https://mars.nasa.gov/news"
browser.visit(url)


In [58]:
# Display mars news title and news paragraph
html = browser.html
soup = bs(html, "lxml")

news_titles = soup.find('div', class_='content_title').text
news_paragraph = soup.find('div', class_='article_teaser_body').text

print(f"News Title : {news_titles}")
print(f"News Content : {news_paragraph} ")


News Title : NASA's InSight Will Study Mars While Standing Still
News Content : The lander's unique science can teach us how planets are born. 


In [59]:
#JPL Mars Space Images-Featured Images
url_images = "https://www.jpl.nasa.gov/spaceimages/?search=&category=featured#submit"
browser.visit(url_images)


In [60]:
#get the base url
from urllib.parse import urlsplit
base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url_images))
print(base_url)


https://www.jpl.nasa.gov/


In [61]:
#Design an xpath selector to grab the images
xpath = "//*[@id=\"page\"]/section[3]/div/ul/li[1]/a/div/div[2]/img"
#use splinter to bring images
results = browser.find_by_xpath(xpath)
img = results[0]
img.click()

In [62]:
#get image url using BeautifulSoup
html_image = browser.html
soup = bs(html_image, 'html.parser')
img_url = soup.find("img", class_="fancybox-image")["src"]
featured_image_url = base_url + img_url
print(featured_image_url)

https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA20318_hires.jpg


In [63]:
#Mars Weather from twitter
weather_info_dict = []
url_weather = "https://twitter.com/marswxreport?lang=en"
browser.visit(url_weather)

html_weather = browser.html
soup = bs(html_weather, "html.parser")

#for mars_weather in soup.find_all('p',class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text"):
#    weather_info_dict.append(mars_weather.text)
time.sleep(3)
tweets = soup.find_all("div",class_='js-tweet-text-container')
mars_weather=""
for tweet in tweets:
    if tweet.p.get_text().startswith("Sol "):
        mars_weather = tweet.p.get_text()
        break
print('Latest Mars Weather: ' +  mars_weather)

Latest Mars Weather: Sol 2174 (2018-09-17), high -22C/-7F, low -68C/-90F, pressure at 8.96 hPa, daylight 05:45-18:01


In [64]:
#Mars Facts
url_facts = "https://space-facts.com/mars/"
fact_df = pd.read_html(url_facts)
mars_fact_df = fact_df[0]
#print(mars_fact_df)
mars_fact_table = mars_fact_df.to_html()
mars_fact_table = mars_fact_table.replace("\n","")
print(mars_fact_table)

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>0</th>      <th>1</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    </

In [67]:
#collection of title and images for Mars hemispheres
hemisphere_image_urls = []
    
hemisphere_names = ['cerberus%20enhanced', 'schiaparelli%20enhanced','syrtis%20major%20enhanced','valles%20marineris%20enhanced']
for hemis in hemisphere_names:
    base ='https://astrogeology.usgs.gov/search/map/Mars/Viking/' + hemis
    browser.visit(base)
    html_hm = browser.html
    soup3 = bs(html_hm,'html.parser')
    #hemisphere_image_urls["image_url"] = soup3.find('img',class_='wide-image')['src']
    imagelist = soup3.find('div',class_="wide-image-wrapper").find_all('li')
    img_url = "https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/"+ hemis.replace("%20","_") + ".tif/full.jpg"
    hemisphere_image_urls.append({"hems_title" : soup3.find('h2',class_='title').text,"image_url": img_url} )
    
for x in hemisphere_image_urls:
    print("image url: " + x['image_url'])
    print("image title: " + x['hems_title'])


image url: https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
image title: Cerberus Hemisphere Enhanced
image url: https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
image title: Schiaparelli Hemisphere Enhanced
image url: https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
image title: Syrtis Major Hemisphere Enhanced
image url: https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
image title: Valles Marineris Hemisphere Enhanced
